<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings


In [2]:
#turn off UserWarnings
warnings.filterwarnings('ignore')

In [3]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(id) FROM VACANCIES
'''

In [6]:
vacancies = pd.read_sql_query(query_3_1, connection).iloc[0][0]
print("Число вакансий в базе: ", vacancies)

Число вакансий в базе:  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''
SELECT COUNT(id) FROM EMPLOYERS
'''

In [8]:
employers = pd.read_sql_query(query_3_2, connection).iloc[0][0]
print("Число работодателей в базе: ", employers)

Число работодателей в базе:  23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
query_3_3 = f'''
SELECT COUNT(id) FROM AREAS
'''

In [10]:
areas = pd.read_sql_query(query_3_3, connection).iloc[0][0]
print("Число регионов: ", areas)

Число регионов:  1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
query_3_4 = f'''
SELECT COUNT(id) FROM INDUSTRIES
'''

In [12]:
industries = pd.read_sql_query(query_3_4, connection).iloc[0][0]
print("Число сфер деятельности компаний: ", industries)

Число сфер деятельности компаний:  294


***

В базе представлены 49 тысяч вакансий. Вакансии размещены 23 тысячами работадателей. Получается на каждого работадателя приходится чуть больше 2 вакансий. У больших компаний число открытых вакансий должно быть больше двух, поэтому можно предположить, что рынок наполняют в основном небольшие организации. **Для уточнения этой гипотезы можно подробнее исслеовать таблицу вакансий**.

Число городов/регионов 1362. Число городов в России меньше  - 1118 на октябрь 2021 года. По всей видимости в списке регионов представлены большинство или все города РФ. А также ряд городов из других стран, в основном ближайших, поскольку они с большей вероятностью будут искать работников на платформе из-за географический и языковых причин.

Число сфер деятельности - 294. Представлен широкий спектр вакансий, не сконцентрированный вокруг узкого направления в дятельности. IT-специалисты находят применение себе в самых разных сферах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
query_4_1 = f'''
SELECT 
    ar.name AS area,
    COUNT(vac.id) AS cnt 
FROM VACANCIES vac
INNER JOIN areas ar ON vac.area_id = ar.id
GROUP BY ar.name
ORDER BY cnt DESC
'''

In [14]:
vac_areas = pd.read_sql_query(query_4_1, connection)

#top 5 for task 4.1
display(vac_areas.iloc[:5])

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
query_4_2 = f'''
SELECT 
    COUNT(id) 
FROM VACANCIES
WHERE salary_from IS NOT NULL
    OR salary_to IS NOT NULL
'''

In [16]:
vac_salary = pd.read_sql_query(query_4_2, connection).iloc[0][0]

print("Число вакансий с данными о зарплате: ", vac_salary)

Число вакансий с данными о зарплате:  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
query_4_3 = f'''
SELECT 
    ROUND(AVG(salary_from)) as AvgMinSalary,
    ROUND(AVG(salary_to)) as AvgMaxSalary
FROM VACANCIES
'''

In [18]:
avg_salary = pd.read_sql_query(query_4_3, connection)

print("Средняя нижняя граница заработной платы: ", int(avg_salary.iloc[0][0]))
print("Средняя верхняя граница заработной платы: ", int(avg_salary.iloc[0][1]))

Средняя нижняя граница заработной платы:  71065
Средняя верхняя граница заработной платы:  110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
query_4_4 = f'''
SELECT 
    schedule,
    employment,
    COUNT(id) as Count
FROM VACANCIES
GROUP BY schedule, employment
ORDER BY Count DESC
'''

In [20]:
shedule_empl = pd.read_sql_query(query_4_4, connection)

#Answer for task 4.4
print("После наиболее популярного сочетания \"Полный день - Полная занятость\" следует вариант:\n")
display(shedule_empl.iloc[[1]])

После наиболее популярного сочетания "Полный день - Полная занятость" следует вариант:



,schedule,employment,count
1,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
query_4_5 = f'''
SELECT 
    experience,
    COUNT(id) as Count
FROM VACANCIES
GROUP BY experience
ORDER BY Count ASC
'''

In [22]:
vac_experience = pd.read_sql_query(query_4_5, connection)

display(vac_experience)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Ранее было определено, что 49 тысяч вакансий распределены в 1362 регионах. Если посмотреть детальнее, то заметно, что треть всех вакансий находятся в пяти крупнейших городах. В большинстве же регионов не наберётся и десятка вакансий. 


**ДАЛЕЕЕЕЕЕ**

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [53]:
query_5_1 = f'''
SELECT 
    emp.name,
    COUNT(vac.id) as VacCount
FROM vacancies vac
INNER JOIN employers emp ON vac.employer_id = emp.id
GROUP BY emp.name
ORDER BY VacCount DESC
LIMIT 5
'''

In [65]:
top_employers = pd.read_sql_query(query_5_1, connection)


print("Первый по числу вакансий работодатель", top_employers.iloc[0][0], "с числом вакансий", top_employers.iloc[0][1])
print("Пятый по числу вакансий работодатель", top_employers.iloc[4][0], "с числом вакансий", top_employers.iloc[4][1])

Первый по числу вакансий работодатель Яндекс с числом вакансий 1933
Пятый по числу вакансий работодатель Газпром нефть с числом вакансий 331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [66]:
query_5_2 = f'''
SELECT
    ar.name,
    COUNT(emp.id) AS EmpCount,
    COUNT(vac.id) AS VacCount 
FROM areas ar
LEFT JOIN vacancies vac ON ar.id = vac.area_id
LEFT JOIN employers emp ON emp.area = ar.id
GROUP BY ar.name
HAVING COUNT(vac.id) = 0
ORDER BY EmpCount DESC
LIMIT 1
'''

In [67]:
top_emp_area = pd.read_sql_query(query_5_2, connection).iloc[0][0]

print("Регион с наибольшим числом работодателей, но без вакансий:", top_emp_area)

Регион с наибольшим числом работодателей, но без вакансий: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
# текст запроса

In [28]:
# результат запроса

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
# текст запроса

In [30]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
# текст запроса

In [32]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
# текст запроса

In [34]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
# код для получения списка городов-милионников

In [36]:
# текст запроса

In [37]:
# результат запроса

***

In [38]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [39]:
# текст запроса

In [40]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [41]:
# текст запроса

In [42]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [43]:
# текст запроса

In [44]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [45]:
# текст запроса

In [46]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
# текст запроса

In [48]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [49]:
# текст запроса

In [50]:
# результат запроса

***

In [51]:
# выводы по предметному анализу

# Общий вывод по проекту

In [52]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования